In [1]:
### Import libraries
import openai
import os
import time
from datetime import datetime
import pandas as pd
import clingo
from clingo.control import Control
import json
from helper import *
from stories import *
from pipelines import *
import psycopg2

In [2]:
now = datetime.now()
date_time = now.strftime("%Y%m%d")

## OpenAI

In [3]:
set_api_key()

## Connect to Database

In [4]:
conn = get_connection()

## Load Projects

In [5]:
projects = get_useful_projects()

## Functions

In [6]:
def save_in_database(conn, prompt1, prompt2, encoding, story, story_1, story_2, version, file):
    cursor = conn.cursor()
    command = "INSERT INTO ne_linebyline (prompt1, prompt2, encoding, story, story_1, story_2, version, file) VALUES ('%(prompt1)s', '%(prompt2)s', '%(encoding)s', '%(story)s', '%(story_1)s', '%(story_2)s', '%(version)s', '%(file)s');" % {"prompt1": prompt1, "prompt2": prompt2, "encoding": encoding, "story": story, "story_1": story_1, "story_2":story_2, "version":version, "file":file}
    try:
        cursor.execute(command)
        conn.commit()
        cursor.close()
    except Exception as err:
        print(command)
        print(f"Unexpected {err=}, {type(err)=}")
        conn.commit()
        cursor.close()
        pass

In [7]:
def quality_check(name_file, lines_file, previous_messages, instance, deep):
    if deep==0:
        return lines_file[-1]
    with open(name_file,"w") as tmp_file:
        for new_line in lines_file:
            tmp_file.write(new_line)
    models, errors, symbols, mistakes = asp_try_5(name_file,instance)
    print(f"Errors {errors}. Messages {mistakes}.")
    new_answer = lines_file[-1]
    if len(mistakes) > 0:
        print("Mistake! Last Message : " + str(previous_messages[-1]))
        answer = get_completion_line("The last rule that you wrote is wrong. The error message is: ###" + str(mistakes) + "###. Can you try again?"
                                    , previous_messages = previous_messages)
        lines_file[-1] = answer
        print("Deep: "+ str(deep) + ". New answer: " +answer)
        new_answer = quality_check(name_file,lines_file,previous_messages, instance, deep-1)
    return lines_file[-1]

## Main : From Script, Line4Line

In [10]:
for project in projects:
    story = project["story"]
    encoding = project["lines"]
    instance = instances_dict[story]
    problem = project["problem"]
    lines = encoding.splitlines()
    new_file = []
    messages = [{"role":"user","content":f"Problem: {problem}"}]
    for line in lines:
        if line == '\n':
            continue
        if len(line.strip()) > 0 and line.strip()[0] == '%':
            print("---------------------------------------------------")
            new_file.append("\n" + line)
            messages.append({"role":"user","content":line})
            print(line)
            answer = get_completion_line(line,previous_messages=messages)
            print(answer)
            new_file.append("\n" + answer)
            messages.append({"role":"assistant","content":answer})
            #new_answer = quality_check(x",new_file,messages,instance,deep=3)
            #new_file[-1] = new_answer
    new_file_name = story + "_v11_SL_" + date_time + ".lp"
    prompt = '''Given the description of a problem, and a commented line, 
                        your task is to translate this commented line in Answer Set Programming. 
                        Just write one or a few predicates or rules. Do not try to solve the whole problem at once.'''
    encoding=' '.join(new_file)
    save_in_database(conn,prompt,'',encoding.replace("'",""),story,'', '', 11, new_file_name)  

---------------------------------------------------
% Define the next position (X+1,Y) if there is no wall between the current position (X,Y) and the next position (X+1,Y), and both (X,Y) and (X+1,Y) are within the labyrinth.       
next_position(X,Y,X1,Y) :- 
    not wall(X,Y,X1,Y), 
    within_labyrinth(X,Y), 
    within_labyrinth(X1,Y), 
    X1 is X+1.
---------------------------------------------------
            % Define the next position (X,Y+1) if there is no wall between the current position (X,Y) and the next position (X,Y+1), and both (X,Y) and (X,Y+1) are within the labyrinth.
next_position(X,Y,X,Y1) :- 
    not wall(X,Y,X,Y1), 
    within_labyrinth(X,Y), 
    within_labyrinth(X,Y1), 
    Y1 is Y+1.
---------------------------------------------------
            % Define the next position (X,Y) to be the current position if (X,Y) is within the labyrinth.
next_position(X,Y,X,Y) :- 
    within_labyrinth(X,Y).
---------------------------------------------------
           % De

In [9]:
new_file

['\n% Define the non-empty cells (X,Y) in the grid, where X ranges from 1 to the number of rows (R) and Y ranges from 1 to the number of columns (C).',
 '\ncell(X,Y) :- X=1..R, Y=1..C.',
 '\n% Define the cells (X,Y) that contain a digit.',
 '\ndigitCell(X,Y,N) :- cell(X,Y), N=0..4.',
 '\n% Define the four possible directions for illumination: up (1,0), down (-1,0), right (0,1), and left (0,-1).',
 '\ndirection(1,0).\ndirection(-1,0).\ndirection(0,1).\ndirection(0,-1).',
 '\n% Define "neighbor" of 6 variables for 2 fields if there is a "delta" distance between them TODO',
 '\nneighbor(X1,Y1,X2,Y2,DX,DY) :- cell(X1,Y1), cell(X2,Y2), direction(DX,DY), X2=X1+DX, Y2=Y1+DY.',
 '\n% Neighbors that are not "digit" are defined as "reach" TODO',
 '\nreach(X1,Y1,X2,Y2) :- neighbor(X1,Y1,X2,Y2,_,_), not digitCell(X2,Y2,_).',
 '\n% Transitive property of relation "reach": all fields that are neighbours in the same direction, have a relation reach',
 '\nreach(X1,Y1,X3,Y3) :- reach(X1,Y1,X2,Y2), reac